In [21]:
import pandas as pd
import numpy as np
import algorithm as alg
from page import Page, PageMode
from utils import gen_sequences, make_requests, declension_faults

student_list = [
    'Дюкин Петр Радиевич',
    'Ермаков Никита Евгеньевич',
    'Ершова Ксения Глебовна',
]

### Список студентов

In [22]:
print('\n'.join(student_list))

Дюкин Петр Радиевич
Ермаков Никита Евгеньевич
Ершова Ксения Глебовна


### Константы $N_i$

In [23]:
for i, student in enumerate(student_list):
    print(f'N{i + 1} =', len(student.split(' ')[1]))

N1 = 4
N2 = 6
N3 = 6


### Последовательность обращений с учетом остатка от деления и исключением нулей

In [24]:
sequences = gen_sequences(student_list)
for i, sequence in enumerate(sequences):
    print(f'P{i + 1} =', '-'.join(map(str, sequence)))

P1 = 1-2-3-1-2-2-2-1-1-2-2-3-2-1
P2 = 2-1-4-3-3-4-4-2-1-3-4-3-3-4-1
P3 = 2-4-3-1-1-3-4-3-4-1-2-4-3-3-1


### Итоговая последовательность обращений

In [25]:
requests = make_requests(student_list)
print(' '.join([f'{r[0]}-{r[1]}' for r in requests]))

1-1 2-2 3-2 1-2 2-1 3-4 1-3 2-4 3-3 1-1 2-3 3-1 1-2 2-3 3-1 1-2 2-4 3-3 1-2 2-4 3-4 1-1 2-2 3-3 1-1 2-1 3-4 1-2 2-3 3-1 1-2 2-4 3-2 1-3 2-3 3-4 1-2 2-3 3-3 1-1 2-4 3-3 2-1 3-1


In [26]:
def show_stat(mode, algorithm, max_size=10):
    Page.mode = mode
    logger = algorithm(requests, max_size)

    df = pd.DataFrame(logger.rows)
    df = df.transpose()

    def highlight_fault(x, color):
        if x.iloc[-2] == 'F':
            result = np.where(x == x[0], 'background-color: crimson; color: white', None)
            result[0] = None
            result[-2] = 'color: red;'
            return result
        else:
            result = np.where(x == x[0], 'background-color: springgreen;', None)
            result[0] = None
            return result
    sdf = df.style.apply(highlight_fault, color='red', axis=1)


    display(sdf)
    print('Количество обращений:', len(requests))
    print('Количество промахов:', logger.faults)
    print('Процент попаданий: ', round(100 - logger.faults / len(requests) * 100, 2), '%', sep='')
    print('Процент промахов: ', round(logger.faults / len(requests) * 100, 2), '%', sep='')

### Оптимальный алгоритм, глобальный поиск

In [27]:
show_stat(PageMode.GLOBAL, alg.opt)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,sequence,memory,,,,,,,,,,faults,replaced
1,1-1,1-1,,,,,,,,,,,
2,2-2,1-1,2-2,,,,,,,,,,
3,3-2,1-1,2-2,3-2,,,,,,,,,
4,1-2,1-1,2-2,3-2,1-2,,,,,,,,
5,2-1,1-1,2-2,3-2,1-2,2-1,,,,,,,
6,3-4,1-1,2-2,3-2,1-2,2-1,3-4,,,,,,
7,1-3,1-1,2-2,3-2,1-2,2-1,3-4,1-3,,,,,
8,2-4,1-1,2-2,3-2,1-2,2-1,3-4,1-3,2-4,,,,
9,3-3,1-1,2-2,3-2,1-2,2-1,3-4,1-3,2-4,3-3,,,


Количество обращений: 44
Количество промахов: 2
Процент попаданий: 95.45%
Процент промахов: 4.55%


### Алгоритм NFU - Выталкивание редко используемой страницы, глобальный поиск

In [28]:
show_stat(PageMode.GLOBAL, alg.nfu)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,sequence,memory,,,,,,,,,,faults,replaced
1,1-1,1-1,,,,,,,,,,,
2,2-2,1-1,2-2,,,,,,,,,,
3,3-2,1-1,2-2,3-2,,,,,,,,,
4,1-2,1-1,2-2,3-2,1-2,,,,,,,,
5,2-1,1-1,2-2,3-2,1-2,2-1,,,,,,,
6,3-4,1-1,2-2,3-2,1-2,2-1,3-4,,,,,,
7,1-3,1-1,2-2,3-2,1-2,2-1,3-4,1-3,,,,,
8,2-4,1-1,2-2,3-2,1-2,2-1,3-4,1-3,2-4,,,,
9,3-3,1-1,2-2,3-2,1-2,2-1,3-4,1-3,2-4,3-3,,,


Количество обращений: 44
Количество промахов: 3
Процент попаданий: 93.18%
Процент промахов: 6.82%


### Оптимальный алгоритм, локальный поиск

In [29]:
show_stat(PageMode.LOCAL, alg.opt)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,sequence,memory,,,,,,,,,,faults,replaced
1,1-1,1-1,,,,,,,,,,,
2,2-2,1-1,2-2,,,,,,,,,,
3,3-2,1-1,2-2,3-2,,,,,,,,,
4,1-2,1-1,2-2,3-2,1-2,,,,,,,,
5,2-1,1-1,2-2,3-2,1-2,2-1,,,,,,,
6,3-4,1-1,2-2,3-2,1-2,2-1,3-4,,,,,,
7,1-3,1-1,2-2,3-2,1-2,2-1,3-4,1-3,,,,,
8,2-4,1-1,2-2,3-2,1-2,2-1,3-4,1-3,2-4,,,,
9,3-3,1-1,2-2,3-2,1-2,2-1,3-4,1-3,2-4,3-3,,,


Количество обращений: 44
Количество промахов: 3
Процент попаданий: 93.18%
Процент промахов: 6.82%


### Алгоритм NFU - Выталкивание редко используемой страницы, локальный поиск

In [30]:
show_stat(PageMode.LOCAL, alg.nfu)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,sequence,memory,,,,,,,,,,faults,replaced
1,1-1,1-1,,,,,,,,,,,
2,2-2,1-1,2-2,,,,,,,,,,
3,3-2,1-1,2-2,3-2,,,,,,,,,
4,1-2,1-1,2-2,3-2,1-2,,,,,,,,
5,2-1,1-1,2-2,3-2,1-2,2-1,,,,,,,
6,3-4,1-1,2-2,3-2,1-2,2-1,3-4,,,,,,
7,1-3,1-1,2-2,3-2,1-2,2-1,3-4,1-3,,,,,
8,2-4,1-1,2-2,3-2,1-2,2-1,3-4,1-3,2-4,,,,
9,3-3,1-1,2-2,3-2,1-2,2-1,3-4,1-3,2-4,3-3,,,


Количество обращений: 44
Количество промахов: 3
Процент попаданий: 93.18%
Процент промахов: 6.82%


### Общая статистика алгоритмов на памяти с 10 страницами

In [42]:
def grid_make_csv(grid_params):
    df = pd.DataFrame(columns=['mode', 'alg', 'max_size', 'faults'])
    for mode in grid_params['mode']:
        for alg_name in grid_params['alg_name']:
            for max_size in grid_params['max_size']:
                Page.mode = mode
                mode_name = mode.value
                logger = getattr(alg, alg_name)(requests, max_size)
                df = df.append({
                    'mode': mode_name,
                    'alg': alg_name,
                    'max_size': max_size,
                    'faults': logger.faults
                }, ignore_index=True)
                
    return df
                
grid_params = {
    'mode': [PageMode.LOCAL, PageMode.GLOBAL],
    'alg_name': ['opt', 'lru', 'fifo', 'nfu'],
    'max_size': [10]
}

df = grid_make_csv(grid_params)
display(df)

,mode,alg,max_size,faults
0,local,opt,10,3
1,local,lru,10,4
2,local,fifo,10,5
3,local,nfu,10,3
4,global,opt,10,2
5,global,lru,10,4
6,global,fifo,10,6
7,global,nfu,10,3


### Вывод
Лучше всего показал себя глобальный оптимальный алгоритм.
### Рейтинг алгоритмов:
1. OPT - Оптимальный
2. NFU - Выталкивание редко используемой страницы
3. LRU - Выталкивание дольше всего не использовавшейся страницы
4. FIFO - Выталкивание первой пришедшей страницы